Let's start with including NuGet package

In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"

Installed Packages Microsoft.ML, 2.0.1 Microsoft.ML.TimeSeries, 2.0.1

Define a class where to store data

In [2]:
using Microsoft.ML.Data;
public class TSP
{
    public TSP() { }

    public TSP(DateTimeOffset timestamp, double value) => (Timestamp, Value) = (timestamp, value);

    [LoadColumn(0)]
    public DateTimeOffset Timestamp { get; }

    [LoadColumn(1)]
    public double Value { get; }
}

Read the file

In [3]:
using System.IO;
var FileName = "data_acme_2020_01_007955_0x20_a.csv";
var FolderName = @"D:\IoTHub\TimeSeriesAnomalyDetectionWithDotNetAndAzure\Data";
var text = await File.ReadAllTextAsync(Path.Combine(FolderName, FileName));
var query = text
    .Split("\r\n") // divide in rows
    .Skip(1) // skip header line
    .Select(xx => xx.Split(';', ','))
    .Where(xx => xx.Length >= 6);
var q = query.Select(e => new TSP(DateTime.Parse(e[5]), double.Parse(e[6])));
var data = q.ToArray();

Let's look inside data

In [4]:
data

index,Timestamp,Value
0,2020-01-01 00:00:00Z,-2118
1,2020-01-01 00:10:00Z,-2102
2,2020-01-01 00:20:00Z,-2101
3,2020-01-01 00:30:00Z,-21
4,2020-01-01 00:40:00Z,-2105
5,2020-01-01 00:50:00Z,-2099
6,2020-01-01 01:00:00Z,-2091
7,2020-01-01 01:10:00Z,-2091
8,2020-01-01 01:20:00Z,-209
9,2020-01-01 01:30:00Z,-2088


Prepare the ML.NET Context

In [5]:
using Microsoft.ML;
using Microsoft.ML.TimeSeries;

var ml = new MLContext();

// Convert data to IDataView.
var dataView = ml.Data.LoadFromEnumerable<TSP>(data);

Define the class where to store the detection

In [6]:
public class SrCnnAnomalyDetection
{
    //vector to hold anomaly detection results. Including isAnomaly, anomalyScore, magnitude, expectedValue, boundaryUnits, upperBoundary and lowerBoundary.
    [VectorType(7)]
    public double[] Prediction { get; set; }
}

Estimate the period

In [7]:
var Period = ml.AnomalyDetection.DetectSeasonality(dataView, nameof(TSP.Value), 200, 0.95);
Period

-1

Detect the anomalies

In [8]:
var options = new SrCnnEntireAnomalyDetectorOptions()
{
    Threshold = 0.5,
    Sensitivity = 80,
    DetectMode = SrCnnDetectMode.AnomalyAndMargin,
    Period = 1

};

var outputDataView = ml.AnomalyDetection.DetectEntireAnomalyBySrCnn(dataView, nameof(SrCnnAnomalyDetection.Prediction), nameof(TSP.Value), options);

// Index   Data    Anomaly AnomalyScore    Mag     ExpectedValue   BoundaryUnit    UpperBoundary   LowerBoundary
var predictions = ml.Data.CreateEnumerable<SrCnnAnomalyDetection>(outputDataView, reuseRowObject: false);


Now draw everything on a graph

In [9]:
#r "nuget: XPlot.Plotly.Interactive"
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

var chart = new Scatter
{
    x = data.Select(xx => xx.Timestamp),
    y = data.Select(xx => xx.Value),
    mode = "lines"
};
var chart2 = new Scatter
{
    x = data.Select(xx => xx.Timestamp),
    y = predictions.Select(xx => xx.Prediction[3]),
    mode = "lines"
};
var detected =
    data
        .Zip(predictions.Select(xx => xx.Prediction[0]))
        .Where(xx => xx.Second == 1) // IsAnomaly
        .Select(xx => xx.First)
        .ToArray();
var chart3 = new Scatter
{
    x = detected.Select(xx => xx.Timestamp),
    y = detected.Select(xx => xx.Value),
    mode = "markers"
};
var plot = Chart.Plot(new[] { chart, chart2, chart3 });
display(plot);

Installed Packages XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `C:\Users\marco\.nuget\packages\xplot.plotly.interactive\4.0.6\interactive-extensions\dotnet\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

<!DOCTYPE html>